In [13]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from datetime import datetime
from joblib import Parallel, delayed
import re
PATH = ChromeDriverManager().install()

In [14]:
lista_streamer = ['ibai', 'kingsleague', 'TheGrefg', 'juansguarnizo', 'SLAKUN10', 'Spursito', 'rivers_gg', 'Perxitaa', 'AdriContreras4', 'gerardromero', 'ikercasillas']

In [15]:
main_url = 'https://twitchtracker.com/'
url =[main_url+ e + '/streams' for e in lista_streamer]
url

['https://twitchtracker.com/ibai/streams',
 'https://twitchtracker.com/kingsleague/streams',
 'https://twitchtracker.com/TheGrefg/streams',
 'https://twitchtracker.com/juansguarnizo/streams',
 'https://twitchtracker.com/SLAKUN10/streams',
 'https://twitchtracker.com/Spursito/streams',
 'https://twitchtracker.com/rivers_gg/streams',
 'https://twitchtracker.com/Perxitaa/streams',
 'https://twitchtracker.com/AdriContreras4/streams',
 'https://twitchtracker.com/gerardromero/streams',
 'https://twitchtracker.com/ikercasillas/streams']

In [16]:
def scrap(url, nombre):
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    
    columns = ['fecha', 'duracion', 'media_viwers', 'maximo_numero_viwers', 'seguidores', 'titulo']
    data = []
    max_pages = 15
    current_page = 1
    while current_page <= max_pages:
        try:
            tabla = driver.find_element(By.ID, 'streams')
            filas = tabla.find_elements(By.TAG_NAME, 'tr')
            for fila in filas:
                columnas = fila.find_elements(By.TAG_NAME, 'td')
                if len(columnas) > 0:
                    fecha_str = columnas[0].text
                    duracion = columnas[1].text.lower()
                    avg_ccv = int(columnas[2].text.replace(",", ""))
                    max_ccv = int(columnas[3].text.replace(",", ""))
                    seguidores = int(columnas[4].text.replace(",", ""))
                    titulo = columnas[6].text.lower()
                    fecha = datetime.strptime(fecha_str, "%d/%b/%Y %H:%M")
                    titulo = titulo.replace(" ", "_")
                    data.append([fecha, duracion, avg_ccv, max_ccv, seguidores, titulo, nombre])
        
            next_page = driver.find_element(By.LINK_TEXT, str(current_page + 1))
            next_page.click()
            current_page += 1
            if current_page > max_pages or not next_page:
                    break
        except:
            break
    driver.close()
    return pd.DataFrame(data, columns=columns + ['nombre'])

lst_df = []
for url, nombre in zip(url, lista_streamer):
    df = scrap(url, nombre)
    lst_df.append(df)

result_df = pd.concat(lst_df, ignore_index=True)
result_df['kings_league'] = result_df['titulo'].apply(lambda x: True if 'king' in x else False)
result_df['duracion'] = result_df['duracion'].str.replace(' hrs', '').astype(float)
result_df.to_csv("base_de_datos.csv", index=False)

C:\Users\jaime\AppData\Local\Temp\ipykernel_8208\707120103.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [17]:
result_df

,fecha,duracion,media_viwers,maximo_numero_viwers,seguidores,titulo,nombre,kings_league
0,2023-08-12 17:06:00,4.0,33682,40583,1622,"mbappé_fuera_de_la_convocatoria,_kepa_al_madri...",ibai,False
1,2023-08-10 17:15:00,4.2,70081,90538,1996,charlando_con_gerard_piqué_|_actualización_sob...,ibai,False
2,2023-08-08 17:07:00,5.3,38359,50372,2361,"charla_sobre_kylian_mbappé,_trivials_y_lolete_...",ibai,False
3,2023-08-07 18:01:00,4.2,54704,66598,2388,pasando_una_agradable_tarde_de_verano_juntos_|...,ibai,False
4,2023-08-06 22:56:00,4.6,39509,49502,2006,stream_nocturno_como_en_los_viejos_tiempos_hab...,ibai,False
...,...,...,...,...,...,...,...,...
2863,2023-01-08 17:38:00,1.8,5420,10862,12503,el_día_de_los_reyes_👑_1k_vs_aniquiladores_#kin...,ikercasillas,True
2864,2023-01-05 12:14:00,1.8,1217,1648,2380,arranca_la_ikerneta_🚀_#justchatting,ikercasillas,False
2865,2023-01-01 16:10:00,1.1,2893,4002,7035,partido_histórico_🟣_1k_fútbol_club_vs_rayo_de_...,ikercasillas,True
2866,2023-01-01 15:31:00,0.2,518,518,0,partido_histórico_🟣_1k_fútbol_club_vs_rayo_de_...,ikercasillas,True
